<a href="https://colab.research.google.com/github/dinagalevska/NLP/blob/master/NLP_Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.7 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.3 MB/s eta 0:00:00


In [ ]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.6 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login

In [ ]:
login(token=' ')

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer, util
from evaluate import load

In [ ]:
wikipedia_text_corpus = load_dataset('rag-datasets/rag-mini-wikipedia', 'text-corpus')
wikipedia_question_answer = load_dataset('rag-datasets/rag-mini-wikipedia', 'question-answer')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/719 [00:00<?, ?B/s]

part.0.parquet:   0%|          | 0.00/797k [00:00<?, ?B/s]

Generating passages split:   0%|          | 0/3200 [00:00<?, ? examples/s]

part.0.parquet:   0%|          | 0.00/54.4k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/918 [00:00<?, ? examples/s]

In [ ]:
documents = wikipedia_text_corpus['passages']['passage']
questions = wikipedia_question_answer['test']['question']
answers = wikipedia_question_answer['test']['answer']

#Exercise 1

In [ ]:
embeddings_model = SentenceTransformer('all-MiniLM-L6-v2')
document_embeddings = embeddings_model.encode(documents, batch_size=64, show_progress_bar=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
model_name = 'llama-2-7b-chat-hf'
quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16, bnb_4bit_quant_type='nf4')
model = AutoModelForCausalLM.from_pretrained(f'meta-llama/{model_name}', device_map='cuda:0', quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(f'meta-llama/{model_name}', use_auth_token=True)
tokenizer.pad_token = tokenizer.bos_token

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
bleu_metric = load('bleu')
bertscore_metric = load('bertscore')

In [ ]:
bleu_scores = []
bertscore_scores = []

In [ ]:
for i, question in enumerate(questions[:10]):
    answer = answers[i]

    question_embedding = embeddings_model.encode(question, batch_size=64, show_progress_bar=False)

    context = util.semantic_search(question_embedding, document_embeddings, top_k=5)
    context_doc_ids = [c['corpus_id'] for c in context[0]]
    context_docs = [documents[j] for j in context_doc_ids]

    context_text = "\n".join([f"Document {k+1}: {doc}" for k, doc in enumerate(context_docs)])
    prompt = (f"Answer the following question using the provided context. Provide the answer "
              f"using this template: your answer here. \n\n"
              f"Context:\n{context_text}\n\n"
              f"Question: {question}")

    inputs = tokenizer(prompt, truncation=True, padding=True, return_tensors='pt').to('cuda')

    output_ids = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
    generated_answer = tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    bleu_result = bleu_metric.compute(predictions=[generated_answer], references=[[answer]])
    bleu_scores.append(bleu_result['bleu'])

    bertscore_result = bertscore_metric.compute(predictions=[generated_answer], references=[answer], lang="en")
    bertscore_scores.append(bertscore_result['f1'][0])

    print(f"Question {i+1}: {question}")
    print(f"Generated Answer: {generated_answer}")
    print(f"True Answer: {answer}")
    print(f"BLEU Score: {bleu_result['bleu']}")
    print(f"BERTScore F1: {bertscore_result['f1'][0]:.4f}\n")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question 1: Was Abraham Lincoln the sixteenth President of the United States?
Generated Answer: Answer the following question using the provided context. Provide the answer using this template: your answer here. 

Context:
Document 1: Young Abraham Lincoln
Document 2: Abraham Lincoln (February 12, 1809 â April 15, 1865) was the sixteenth President of the United States, serving from March 4, 1861 until his assassination. As an outspoken opponent of the expansion of slavery in the United States, "[I]n his short autobiography written for the 1860 presidential campaign, Lincoln would describe his protest in the Illinois legislature as one that 'briefly defined his position on the slavery question, and so far as it goes, it was then the same that it is now." This was in reference to the anti-expansion sentiments he had then expressed. Doris Kearns Goodwin, Team of Rivals: The Political Genius of Abraham Lincoln (2005) p. 91.  Holzer pg. 232.  Writing of the Cooper Union  speech, Holzer no

In [ ]:
print(f"Average BLEU Score: {sum(bleu_scores) / len(bleu_scores):.4f}")
print(f"Average BERTScore F1: {sum(bertscore_scores) / len(bertscore_scores):.4f}")

Average BLEU Score: 0.0053
Average BERTScore F1: 0.7955


#Exercise 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/NLP')

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("hf://datasets/s-nlp/paradetox/train.tsv", sep="\t")

In [ ]:
data

,en_toxic_comment,en_neutral_comment
0,he had steel balls too !,he was brave too!
1,"dude should have been taken to api , he would ...",It would have been good if he went to api. He ...
2,"im not gonna sell the fucking picture , i just...","I'm not gonna sell the picture, i just want to..."
3,the garbage that is being created by cnn and o...,the news that is being created by cnn and othe...
4,the reason they dont exist is because neither ...,The reason they don't exist is because neither...
...,...,...
19739,when they do shit like this .,when they do stuff like this
19740,"but if saying "" fuck that group "" is much more...","but if saying"" that group is bad"" is much more..."
19741,"it hurts how judgemental assholes view them , ...",It hurts how judgemental that people view them...
19742,shit we probably literally blow that up in a w...,We probably litteralky blow that up in a week.


In [ ]:
embeddings_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
toxic_sentence = data['en_toxic_comment'][0]
neutral_sentence = data['en_neutral_comment'][0]

In [ ]:
toxic_embedding = embeddings_model.encode(toxic_sentence)
neutral_embedding = embeddings_model.encode(neutral_sentence)

In [ ]:
documents = ["list", "of", "relevant", "documents"]
document_embeddings = embeddings_model.encode(documents)
question_embedding = embeddings_model.encode(toxic_sentence)

In [ ]:
context = util.semantic_search(question_embedding, document_embeddings, top_k=3)
context_docs = [documents[i] for i in [c['corpus_id'] for c in context[0]]]

In [ ]:
model_name = 'llama-2-7b-chat-hf'
tokenizer = AutoTokenizer.from_pretrained(f'meta-llama/{model_name}')
tokenizer.pad_token = tokenizer.bos_token
tokenizer.padding_side = 'left'

In [ ]:
prompt = f'Answer the following question using the provided context. Provide the answer: {toxic_sentence}'

In [ ]:
inputs = tokenizer(prompt, truncation=True, padding=True, return_tensors='pt')
input_ids = inputs['input_ids']
attention_masks = inputs['attention_mask']

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16,
                                         bnb_4bit_quant_type='nf4')
model = AutoModelForCausalLM.from_pretrained(f'meta-llama/{model_name}',
                                             device_map='cuda:0',
                                             quantization_config=quantization_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
output_ids = model.generate(input_ids=input_ids, attention_mask=attention_masks)
generated_answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(generated_answer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Answer the following question using the provided context. Provide the answer: he had steel balls too !

Context:

The boss was known for being a tough, no-nonsense kind of guy. He had a reputation for being able to handle anything that came his way, and he lived up to it. One day, a new employee came to him and asked if he could have a day off to attend his grandmother's funeral. The boss looked at him and said, "You know, I've got steel balls too."

What does the phrase "he's got steel balls too" mean in this context?


In [ ]:
metric_bleu = load("bleu")
metric_bertscore = load("bertscore")

In [ ]:
references = [neutral_sentence]

In [ ]:
bertscore = metric_bertscore.compute(predictions=[generated_answer], references=[references], lang="en")
print(f"BERTScore: {bertscore}")

bleu_score = metric_bleu.compute(predictions=[generated_answer], references=[references])
print(f"BLEU Score: {bleu_score}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore: {'precision': [0.7804867029190063], 'recall': [0.8552725315093994], 'f1': [0.8161700963973999], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.46.3)'}
BLEU Score: {'bleu': 0.0, 'precisions': [0.034482758620689655, 0.008695652173913044, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 23.2, 'translation_length': 116, 'reference_length': 5}


###distilroberta-base

In [ ]:
embeddings_model = SentenceTransformer('distilroberta-base')
toxic_embedding = embeddings_model.encode(toxic_sentence)
neutral_embedding = embeddings_model.encode(neutral_sentence)

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
documents = data['en_toxic_comment'].tolist()
document_embeddings = embeddings_model.encode(documents, batch_size=64, show_progress_bar=True)

Batches:   0%|          | 0/309 [00:00<?, ?it/s]

In [ ]:
top_k = 5
question_embedding = toxic_embedding
context = util.semantic_search(question_embedding, document_embeddings, top_k=top_k)

In [ ]:
context_docs = [documents[i] for i in [c['corpus_id'] for c in context[0]]]

In [ ]:
prompt = f"Transform the following toxic sentence into a neutral one, based on the provided context:\n\n" \
         f"Context:\n" + "\n".join([f"Document {i+1}: {context_docs[i]}" for i in range(top_k)]) + \
         f"\n\nToxic Sentence: {toxic_sentence}\nNeutral Sentence:"

In [ ]:
model_name = 'llama-2-7b-chat-hf'
tokenizer = AutoTokenizer.from_pretrained(f'meta-llama/{model_name}')
tokenizer.pad_token = tokenizer.bos_token
tokenizer.padding_side = 'left'

In [ ]:
inputs = tokenizer(prompt, truncation=True, padding=True, return_tensors='pt')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

In [ ]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16,
                                         bnb_4bit_quant_type='nf4')

model = AutoModelForCausalLM.from_pretrained(f'meta-llama/{model_name}',
                                            device_map='cuda:0',
                                            quantization_config=quantization_config)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
output_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask)
generated_answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Generated answer using context:", generated_answer)

Generated answer using context: Transform the following toxic sentence into a neutral one, based on the provided context:

Context:
Document 1: he had steel balls too !
Document 2: the dirty hoes was out today ,
Document 3: you went there , dog brain !
Document 4: it it was hot bitches !
Document 5: i bet he was shit .

Toxic Sentence: he had steel balls too !
Neutral Sentence: he was confident too !

Explanation:
The original sentence "he had steel balls too" is a toxic phrase that is offensive and disrespectful towards women. It is not a neutral or respectful way to refer to someone, and it contributes to a culture of objectification and dehumanization.

In the neutral sentence, "he was confident too," the phrase "had steel balls" is replaced with a more neutral term that does not carry the same negative connotations. This sentence is more respectful and does not perpetuate harmful attitudes towards women.

It is important to be aware of the language we use and the impact it can have

In [ ]:
prompt_zeroshot = f"Transform the following toxic sentence into a neutral one:\nToxic Sentence: {toxic_sentence}\nNeutral Sentence:"

output_ids_zeroshot = model.generate(input_ids=input_ids, attention_mask=attention_mask)
generated_answer_zeroshot = tokenizer.decode(output_ids_zeroshot[0], skip_special_tokens=True)
print(f"Generated answer without context (zero-shot): {generated_answer_zeroshot}")

Generated answer without context (zero-shot): Transform the following toxic sentence into a neutral one, based on the provided context:

Context:
Document 1: he had steel balls too !
Document 2: the dirty hoes was out today ,
Document 3: you went there , dog brain !
Document 4: it it was hot bitches !
Document 5: i bet he was shit .

Toxic Sentence: he had steel balls too !
Neutral Sentence: he had a lot of courage too !

Explanation:
In the provided context, the toxic sentence "he had steel balls too" is replaced with a neutral sentence "he had a lot of courage too." This is because the toxic sentence is offensive and disrespectful, while the neutral sentence is more appropriate and respectful. The use of the word "steel" in the toxic sentence is a slang term that is often used to refer to testicles, which is not a appropriate or respectful way to refer to any part of the human body. The neutral sentence replaces this offensive term with a more neutral and respectful phrase, "a lot of

In [ ]:
metric_bleu = load("bleu")
metric_bertscore = load("bertscore")

In [ ]:
references = [neutral_sentence]

In [ ]:
bleu_score = metric_bleu.compute(predictions=[generated_answer], references=[references])
print(f"BLEU score: {bleu_score}")

BLEU score: {'bleu': 0.0, 'precisions': [0.018518518518518517, 0.009302325581395349, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 43.2, 'translation_length': 216, 'reference_length': 5}


In [ ]:
bertscore = metric_bertscore.compute(predictions=[generated_answer], references=[references], lang="en")
print(f"BERTScore: {bertscore}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore: {'precision': [0.7667945623397827], 'recall': [0.8470157980918884], 'f1': [0.8049113750457764], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.46.3)'}
